# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [230]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [231]:
# your code here
orders = pd.read_csv('Orders.csv')
# Turn everying to snake_case
orders.columns = list(map(lambda x: x.lower(), orders.columns))
orders.columns = orders.columns.str.replace(' ','_')

# dropping old index column
orders = orders.drop('unnamed:_0',axis=1) 

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [232]:
#Problem 1: How to aggregate the  `amount_spent` for unique customers?
Q1 = orders.groupby(['customerid']).agg({'amount_spent':sum})

#Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?
q95 =Q1.quantile(0.95)[0]
q75 =Q1.quantile(0.75)[0]
# Q1[Q1['amount_spent']>qXX]]

#Problem 3: How to label selected customers as "VIP" or "Preferred"?
def label_customers(customer,q75,q95):
    if customer[0]>q95:
        return 'VIP'
    elif customer[0]>q75:
        return 'Preferred'
    else: 
        return 'Unrated'

Q1['customer_type'] = Q1.apply(lambda x: label_customers(x,q75,q95),axis=1)
Q1


,amount_spent,customer_type
customerid,,
12346,77183.60,VIP
12347,4310.00,Preferred
12348,1797.24,Preferred
12349,1757.55,Preferred
12350,334.40,Unrated
...,...,...
18280,180.60,Unrated
18281,80.82,Unrated
18282,178.05,Unrated


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [233]:
# I'll merge my previous table with a new one with the countries, instead of creating again the column customer_type
preQ1 = Q1.drop('amount_spent',axis=1)
preQ2 =  orders.groupby(['customerid','country'],as_index=False).agg({'amount_spent':sum})
Q2 = pd.merge(left = preQ2, right = preQ1, how = 'inner', right_index = True, left_on='customerid')
Q2

,customerid,country,amount_spent,customer_type
0,12346,United Kingdom,77183.60,VIP
1,12347,Iceland,4310.00,Preferred
2,12348,Finland,1797.24,Preferred
3,12349,Italy,1757.55,Preferred
4,12350,Norway,334.40,Unrated
...,...,...,...,...
4342,18280,United Kingdom,180.60,Unrated
4343,18281,United Kingdom,80.82,Unrated
4344,18282,United Kingdom,178.05,Unrated
4345,18283,United Kingdom,2094.88,Preferred


In [234]:
# Filtering for VIP customers
Q2 = Q2[Q2['customer_type']=='VIP']
# Grouping by country, customer_type and counting them:
Q2 = Q2.groupby(['country','customer_type'],as_index=False).agg(n_customers=('customerid',len))
Q2.sort_values('n_customers',ascending=False)

,country,customer_type,n_customers
17,United Kingdom,VIP,177
8,Germany,VIP,10
7,France,VIP,9
16,Switzerland,VIP,3
14,Spain,VIP,2
12,Portugal,VIP,2
0,Australia,VIP,2
5,EIRE,VIP,2
9,Japan,VIP,2
6,Finland,VIP,1


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [235]:
# I'll merge my previous table with a new one with the countries, instead of creating again the column customer_type
preQ1 = Q1.drop('amount_spent',axis=1)
preQ2 =  orders.groupby(['customerid','country'],as_index=False).agg({'amount_spent':sum})
Q2 = pd.merge(left = preQ2, right = preQ1, how = 'inner', right_index = True, left_on='customerid')
Q2

,customerid,country,amount_spent,customer_type
0,12346,United Kingdom,77183.60,VIP
1,12347,Iceland,4310.00,Preferred
2,12348,Finland,1797.24,Preferred
3,12349,Italy,1757.55,Preferred
4,12350,Norway,334.40,Unrated
...,...,...,...,...
4342,18280,United Kingdom,180.60,Unrated
4343,18281,United Kingdom,80.82,Unrated
4344,18282,United Kingdom,178.05,Unrated
4345,18283,United Kingdom,2094.88,Preferred


In [236]:
# Filtering for VIP and Prefererred customers
Q2 = Q2[(Q2['customer_type']=='VIP')|(Q2['customer_type']=='Preferred')]
# Grouping by country and counting them:
Q2 = Q2.groupby(['country'],as_index=False).agg(n_customers=('customerid',len))
Q2.sort_values('n_customers',ascending=False)

,country,n_customers
26,United Kingdom,932
10,Germany,39
9,France,29
2,Belgium,12
25,Switzerland,9
23,Spain,9
21,Portugal,7
19,Norway,7
14,Italy,5
8,Finland,5
